## Control and Identification in Julia

### DC motor

In this tutorial we will model, identify and control the speed of a DC motor.
We will cover the following topics

- System modelling
  - Representations
  - Simulation
  - Saving/loading data
  
  
- System identification
  - Transfer function estimation
  - State space estimation


- Control design
  - Root locus
  - Frequency domain (lead lag)
  - Pole placement

This tutorial is adapted from
[Control Tutorials for Matlab and Simulink (CTMS)](http://ctms.engin.umich.edu/CTMS/index.php?example=MotorSpeed&section=SystemModeling)


### Installation

In [ ]:
Pkg.clone("https://github.com/JuliaSystems/SystemsBase.jl")
Pkg.clone("https://github.com/JuliaSystems/ControlToolbox.jl")
Pkg.clone("https://github.com/JuliaSystems/IdentificationToolbox.jl")

In [ ]:
Pkg.update()
Pkg.checkout("RationalFunctions")
Pkg.checkout("PolynomialMatrices")

In [ ]:
using SystemsBase
using ControlToolbox
using IdentificationToolbox

In [ ]:
using Plots
pyplot()

### JuliaSystems ecosystem

What we aim for...


<img src="assets/ecosystem.svg" width="800">

### Dependencies

What we rely on...

<img src="assets/juliasystems.svg" width="800">

### System modelling

#### Physical setup

For this example, we will assume that the input of the system is the voltage source (V) applied to the motor's armature, while the output is the rotational speed of the shaft d(theta)/dt. The rotor and shaft are assumed to be rigid. We further assume a viscous friction model, that is, the friction torque is proportional to shaft angular velocity.


![title](figures/motor.png)

Figure taken from [CTMS](http://ctms.engin.umich.edu/CTMS/index.php?example=MotorSpeed&section=SystemModeling).

The physical constants are:

| parameter     | description                     |  value         |
| :------------ | :------------------------------ | :------------- |
| J             | moment of inertia of the rotor  | 0.01 kg.m^2    |
| b             | motor viscous friction constant | 0.1 N.m.s      |
| Ke            | electromotive force constant    | 0.01 V/rad/sec |
| Kt            | motor torque constant           | 0.01 N.m/Amp   |
| R             | electric resistance             | 1 Ohm          |
| L             | electric inductance             | 0.5 H          |


#### System equations

Motor torque is proportional to the armature current $i$ by a constant factor $Kt$

$$
T=K_{t}
$$

The back electromotive force, $e$, is proportional to the angular velocity of the shaft by a constant factor Ke.

$$
e = K_{e} \dot{\theta}
$$

In SI units, the motor torque and back electromotive force constant are equal, that is, $Kt = Ke$;
therefore, we will use $K$ to represent both the motor torque constant and the back electromotive force constant.

From the figure above, we can derive the following governing equations based on Newton's 2nd law
and Kirchhoff's voltage law.

$$
J\ddot{\theta} + b \dot{\theta} = K i
$$

$$ L \frac{di}{dt} + Ri = V - K\dot{\theta}$$

### System representations

#### Transfer function

Applying the Laplace transform, the above modeling equations can be expressed in terms of the Laplace variable s.

$$ s(Js + b)\Theta(s) = KI(s) $$

$$ (Ls + R)I(s) = V(s) - Ks\Theta(s) $$

We arrive at the following open-loop transfer function by eliminating I(s) between the two above equations, where the rotational speed is considered the output and the armature voltage is considered the input.

$$
P(s) = \frac {\dot{\Theta}(s)}{V(s)} = \frac{K}{(Js + b)(Ls + R) + K^2} \qquad \left[ \frac{rad/sec}{V} \right]
$$

In [ ]:
s = tf([1,0], 1)

J = 0.01
b = 0.1
K = 0.01
R = 1.
L = 0.5

Ptf = K/((J*s+b)*(L*s+R)+K^2)

#### State-space

In state-space form, the governing equations above can be expressed by choosing the rotational speed and electric current as the state variables. 

$$
\frac{d}{dt}
\begin{bmatrix} \dot{\theta} \\ i \end{bmatrix}
    =
\begin{bmatrix}
    -\frac{b}{J} & \frac{K}{J} \\
    -\frac{K}{L} & -\frac{R}{L}
\end{bmatrix}
\begin{bmatrix} \dot{\theta} \\  i \end{bmatrix}  +
\begin{bmatrix} 0 \\ \frac{1}{L} \end{bmatrix} V
$$

$$
y =
\begin{bmatrix} 1 & 0 \end{bmatrix}
\begin{bmatrix} \dot{\theta} \\ i \end{bmatrix}
$$

In [ ]:
A = [-b/J   K/J;
    -K/L   -R/L]
B = [0 1/L].'
C = [1. 0]
D = 0.

Pss = ss(A,B,C,D)

#### Design requirements

For a unit step command in motor speed, the control system's output should meet the following requirements.

- Settling time less than 2 seconds
- Overshoot less than 5%
- Steady-state error less than 1%

#### Open loop characteristics

In [ ]:
plot(pzmap(Pss), xlim = (-10, 10), ylim = (-10,10))

Because of the dominant pole, the open loop system behaves very much like the first order system

$$
\frac{0.1}{0.5s +1}
$$

In [ ]:
Psimple = tf([0.1],[0.5, 1])

plot(step(Pss, (0., 10.)))
plot!(step(Psimple, (0., 10.)), label = "Psimple")

### Simulation

We will now generate some data that can be used for identification

In [ ]:
sln = simulate(Pss, (0., 40.), input = (t,x)->sin(0.4t)+sin(t) + sin(3t) + sin(5t), saveat = 0.05)
writecsv("collected-data.csv", [sln.t sln.y sln.u])

In [ ]:
plot(sln, inputs = [1])

### System identification

#### Loading data

In [ ]:
data, header = readcsv("collected-data.csv", header=true)
N   = size(data,1)

t   = data[:,1]
u   = data[:,3]
y0  = data[:,2]
y   = y0 + 0.3*randn(N) # add some noise

plot(t, [y u y0], layout = (2,1), label = ["y" "u" "y0"] )

#### Split data

Split data into identification and validation part

In [ ]:
Ni  = 400

ti  = t[1:Ni]
ui  = u[1:Ni]
yi0 = y0[1:Ni]
yi  = y[1:Ni]

tv  = t[Ni+1:end]
uv  = u[Ni+1:end]
yv0 = y0[Ni+1:end]
yv  = y[Ni+1:end]

Ts    = 0.5
datai = iddata(yi,ui,Ts)
datav = iddata(yv,uv,Ts)

#### Transfer function identification

We identify a transfer function model of order 2
$$
    P(z) = \frac{b_0 z^{-1} + b_1 z^{-2}}{1 + f_1 z^{-1} + f_2 z^{-2}}
$$

We asssume that we know that the noise is white noise.
This is an Output-Error model.

In [ ]:
model   = OE(2,2,1)

Ptfinit = morsm(datai, model)
Ptfid   = pem(datai, Ptfinit)
Pss2    = ss(Ptfid)

#### Compare prediction on validation data

In [ ]:
ŷ = predict(datav, Ptfid)[:]
plot(tv, [yv0 yv ŷ], layout = (2,1), label = ["\$y_{0}\$" "y" "\$\\hat{y}\$"])

#### State-space identification

**Note.** State space identification (n4sid) is work in progress

In [ ]:
# Pssid = n4sid(datai, 2)
# Pssid = ss(Pssid);

#### Discretize true system

In [ ]:
Ts = 0.05
Pssd, xomap = c2d(Pss, Ts)
Pssd

In [ ]:
plot(simulate(Pssd, (0.,10.), input = (t,x)->sin(t) + sin(3t) + sin(5t)))

#### Discretization continued

In [ ]:
?c2d

In [ ]:
Pssd, xomap = c2d(Pss, Ts, Discretization.Bilinear()) # ZOH by default

discretizer = Discretization.ZOH()

Pssd, xomap = discretizer(Pss, Ts)
Pssd

### Root locus

#### P-controller

We investigate the behaviour of a simple proportional controller in the feedback loop:

![title](figures/MotorSpeed_ControlRootLocus_feedback_motors.png)

(Figure taken from [CTMS](http://ctms.engin.umich.edu/CTMS/index.php?example=MotorSpeed&section=ControlRootLocus)),
where
$$
C(s) = c
$$

In [ ]:
rl = rootlocus(Ptf)
animate(rl, "rootlocus.gif")

To make the system faster without too much overshoot we try to place the poles at $-6\pm 2j$

In [ ]:
c = 10
rl(c)

In [ ]:
Pcl = feedback(Ptf*c,1)
plot(step(ss(Pcl), (0., 10.)))

#### Lag compensator

In the above we saw that the overshoot and settling time criteria were met with the proportional controller,
but the steady-state error requirement was not.
A lag compensator is one type of controller known to be able to reduce steady-state error.

$$ C(s) = \frac {s + 1} { s + 0.01 } $$

In [ ]:
C = tf([1, 1.], [1., .01])
rl = rootlocus(Ptf*C)

plotly()
plot(rl)

We pick the gain 44.

In [ ]:
pyplot()
Pcl = feedback(44*Ptf*C,1)
plot(step(ss(Pcl), (0., 10.)))

### Frequency domain

#### Frequency evaluation

Here we will design a lag compensator using frequency domain methods.

In [ ]:
# Pss(s = 10)
Pss(10)

In [ ]:
# Pss(s = iω)
Pss(ω = 10)

In [ ]:
# Pss(s = iω)
freqresp(Ptf, 10)

#### Nyquist curve

In [ ]:
ω = logspace(-6, 6, 1000)
plot(nyquist(Pss, ω))

#### Bode diagram

In [ ]:
br = bode(Pss, ω)
plot(br)

A phase margin of 60 degrees is achieved for a crossover frequency of approximately 10 rad/sec. The gain should be increased approximately 40 dB.

In [ ]:
C = 1/0.01
Pcl = feedback(C*Ptf,1)
plot(step(Pcl, (0., 10.)))

Note that the settling time is fast enough, but the overshoot and the steady-state error are too high. But can be fixed with a lag compensator.

#### Lag compensator

Consider the following lag compensator:

$$ C(s) = \frac {s + 1} { s + 0.01 } $$

This lag compensator has a DC gain of 1/0.01 = 100 and will only add minor phase to around the crossover frequency of 10 rad/sec.

In [ ]:
C = 45*(s + 1)/(s + 0.01)
Pcl = feedback(C*Ptf,1)
plot(step(Pcl, (0., 10.)))

### Pole placement

#### Check controllability

Since both of the state variables in our problem are easy to measure (simply add an ammeter for current and a tachometer for the speed), we can design a full-state feedback controller for the system without worrying about having to add an observer. The control law for a full-state feedback system has the form 

$$ u = r - Kc x $$ 

and the associated schematic is shown below.

![title](figures/statefeedback_motors.png)

Figure taken from [CTSM](http://ctms.engin.umich.edu/CTMS/index.php?example=MotorSpeed&section=ControlStateSpace).

In [ ]:
A = [-b/J   K/J
    -K/L   -R/L]
B = [0 1/L].'
C = [1. 0]
D = 0.
Pss = ss(A,B,C,D)

sys_order = numstates(Pss)
sys_rank  = rank(controllability(Pss))

sys_order == sys_rank

We try to place the poles on the real axis. 

#### Place poles

In [ ]:
Kc = place(A,B,[-6,-6])

In [ ]:
Pcl = ss(A-B*Kc,B,C,D)

In [ ]:
poles(Pcl)

In [ ]:
?place

In [ ]:
plot(step(Pcl, (0., 10.)))

#### Correct dc gain

To get the the correct steady state, we calculate the dcgain of the closed loop system and set the final control law to 

$$ u = \bar{N} r - Kc x $$ 

where $\bar{N}$ = 1/dcgain(Pcl).

In [ ]:
N̄ = 1/dcgain(Pcl)
Pcl = N̄*Pcl
plot(step(Pcl, (0., 10.)))

### Advanced topics

#### Simulation

In [ ]:
sim = simulate(Pcl, (0.,10.), tstops = [1,4]) do t,x
    1 ≤ t ≤ 4 ? one(t) : zero(t)
end
plot(sim, inputs = [1], states = [1,2])

In [ ]:
mysignal = Signals.Step(1.,2.) - Signals.Ramp(3.,2) + Signals.Ramp(4., 3) -
    Signals.Ramp(6.,1.) - Signals.Step(8.0,2.)
sim = simulate(Pcl, (0.,10.), input = mysignal)
plot(sim, inputs = [1], states = [1,2])

#### Identification

In [ ]:
function cost(data, model, x, options)
  y_est = predict(data, model, x, options)
  value(options.loss_function, data.y, y_est)
end

function pem(data, model, x0, options)
    costf = x->cost(data, model, x, options)
    opt = optimize(costf, x0, Newton(), options)
    give_me_idmodel(model, opt, options)
end

In [ ]:
Ptf = pem(datai, Ptfinit, IdOptions(loss_function = HuberLoss()))
ss(Ptf)

### Implemented and planned features

#### Control and base functionality

| Functionality                  | Implementation status      |
| :----------------------------- | :------------------------: |
|                                |                            |
| Representations                |                            |
| state-space                    | yes                        |
| transfer function              | yes                        |
| matrix fraction description    | yes                        |
| models with time delays        | no                         |
|                                |                            |
| Interconnections               |                            |
| series                         | yes                        |
| parallel                       | yes                        |
| feedback                       | yes                        |
| lft                            | no                         |
|                                |                            |
| Simulation                     |                            |
| simulate transfer function     | yes                        |
| simulate state-space           | yes                        |
| simulate MFD                   | work in progress           |
| step                           | yes                        |
| ramp                           | yes                        |
| sinusoidal                     | work in progress           |
| PRBS                           | work in progress           |
| impulse                        | work in progress           |
|                                |                            |
| Frequency domain analysis      |                            |
| Bode                           | yes                        |
| Nyquist                        | yes                        |
| Nichols                        | no                         |
| frequency evaluations          | yes                        |
| norm                           | future work                |
|                                |                            |
| Continuous-discrete conversion |                            |
| c2d                            | yes                        |
| d2c                            | yes                        |
| resampling discrete model      | no                         |
|                                |                            |
| Model reduction                |                            |
| minimal realization            | yes                        |
| balanced reduction             | no                         |
|                                |                            |
| Stability analysis             |                            |
| poles                          | yes                        |
| zeros                          | yes                        |
| transmission zeros             | yes                        |
| pole zero map                  | yes                        |
|                                |                            |
| Control system design          |                            |
| PID design                     | future work                |
| root locus                     | yes                        |
| pole placement                 | yes                        |
| LQR/LQG                        | future work                |
|                                |                            |
| Matrix computations            |                            |
| controllability/observability  | yes                        |
| gramians                       | yes                        |
| Riccati equation solvers       | yes (could use an upgrade) |
| Lyapunov equation solvers      | yes (could use an upgrade) | 


| Extensions                     | Implementation status      |
| :----------------------------- | :------------------------: |
|                                |                            |
| Robust control                 | no                         |
| Optimal control                | no                         |
| Model predictive control       | work in progress           |

#### System identification

| Functionality                   | Implementation status      |
| :------------------------------ | :------------------------: |
|                                 |                            |
| Data processing                 |                            |
| detrend                         | yes                        |
| fft                             | yes (Julia)                |
| ifft                            | yes (Julia)                |
|                                 |                            |
| Input generation                |                            |
| PRBS                            | work in progress           |
| random phase multi sine         | work in progress           |
| input design                    | no                         |
|                                 |                            |
| Polynomial models               |                            |
| prediction error method         | yes                        |
| instrumental variable           | work in progress           |
|                                 |                            |
| State-space                     |                            |
| n4sid                           | work in progress           |
|                                 |                            |
| Frequency domain                |                            |
| etfe                            | no                         |
| spectral analysis               | no                         |
|                                 |                            |
| Model validation                |                            |
| compare models                  | work in progress           |
| residual analysis               | no                         |
| parameter covariances           | no                         |
| confidence regions              | no                         |
|                                 |                            |
| Model selection                 |                            |
| Akaike's final prediction error | no                         |
| Akaike's Information Criterion  | no                         |
|                                 |                            |
| Online estimation               |                            |
| recursive OE etc.               | no                         |


| Extensions                     | Implementation status      |
| :----------------------------- | :------------------------: |
|                                |                            |
| Nonlinear models               | no                         |
| Bayesian estimation            | no                         |